# Install

In [ ]:
!git clone https://github.com/kopyl/PixArt-alpha.git

In [ ]:
%cd PixArt-alpha

In [ ]:
!pip install torch==2.0.0+cu117 torchvision==0.15.1+cu117 torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cu117
!pip install -r requirements.txt
!pip install wandb

In [2]:
%cd ../PixArt-alpha

/n/home12/binxuwang/Github/DiffusionObjectRelation/PixArt-alpha


## Download model

In [3]:
!python tools/download.py --model_names "PixArt-XL-2-512x512.pth"

^C
Traceback (most recent call last):
  File "/n/home12/binxuwang/Github/DiffusionObjectRelation/PixArt-alpha/tools/download.py", line 10, in <module>
    from torchvision.datasets.utils import download_url
  File "/n/home12/binxuwang/.conda/envs/torch2/lib/python3.10/site-packages/torchvision/__init__.py", line 6, in <module>
    from torchvision import _meta_registrations, datasets, io, models, ops, transforms, utils
  File "/n/home12/binxuwang/.conda/envs/torch2/lib/python3.10/site-packages/torchvision/datasets/__init__.py", line 36, in <module>
    from .kitti import Kitti
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1002, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 945, in _find_spec
  File "<frozen importlib._bootstrap_external>", line 1439, in find_spec
  File "<frozen importlib._bootstrap_external>", line 1411, in _get_spec
  File "<frozen importlib._bootstrap_external>", line 1577,

## Make dataset out of Hugginggface dataset

In [8]:
import os
from tqdm.notebook import tqdm
from datasets import load_dataset
import json

In [9]:
# dataset = load_dataset("lambdalabs/pokemon-blip-captions")
from datasets import load_dataset

dataset = load_dataset("reach-vb/pokemon-blip-captions")

In [12]:
!echo $STORE_DIR/DL_Projects/PixArt

/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt


In [11]:
!mkdir $STORE_DIR/DL_Projects/PixArt

In [10]:
from os.path import join
root_dir = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/pixart-pokemon"
images_dir = "images"
captions_dir = "captions"

images_dir_absolute = join(root_dir, images_dir)
captions_dir_absolute = join(root_dir, captions_dir)

if not os.path.exists(root_dir):
    os.makedirs(join(root_dir, images_dir))

if not os.path.exists(join(root_dir, images_dir)):
    os.makedirs(join(root_dir, images_dir))
if not os.path.exists(join(root_dir, captions_dir)):
    os.makedirs(join(root_dir, captions_dir))

image_format = "png"
json_name = "partition/data_info.json"
if not os.path.exists(join(root_dir, "partition")):
    os.makedirs(join(root_dir, "partition"))

absolute_json_name = join(root_dir, json_name)
data_info = []

order = 0
for item in tqdm(dataset["train"]): 
    image = item["image"]
    image.save(f"{images_dir_absolute}/{order}.{image_format}")
    with open(f"{captions_dir_absolute}/{order}.txt", "w") as text_file:
        text_file.write(item["text"])
    
    width, height = 512, 512
    ratio = 1
    data_info.append({
        "height": height,
        "width": width,
        "ratio": ratio,
        "path": f"{order}.{image_format}", # images/ # if there is images/ in the path it will cause error in InternalData.py L55, L56. 
        "prompt": item["text"],
    })
        
    order += 1

with open(absolute_json_name, "w") as json_file:
    json.dump(data_info, json_file)

  0%|          | 0/833 [00:00<?, ?it/s]

## Extract features

In [1]:
%cd ../PixArt-alpha

/n/home12/binxuwang/Github/DiffusionObjectRelation/PixArt-alpha


In [11]:
!python tools/extract_features.py \
    --img_size 512 \
    --dataset_root $STORE_DIR"/DL_Projects/PixArt/pixart-pokemon" \
    --json_path $STORE_DIR"/DL_Projects/PixArt/pixart-pokemon/partition/data_info.json" \
    --t5_save_root $STORE_DIR"/DL_Projects/PixArt/pixart-pokemon/caption_feature_wmask" \
    --vae_save_root $STORE_DIR"/DL_Projects/PixArt/pixart-pokemon/img_vae_features" \
    --pretrained_models_dir $STORE_DIR"/DL_Projects/PixArt/output/pretrained_models"

/n/home12/binxuwang/.conda/envs/torch2/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/n/home12/binxuwang/.conda/envs/torch2/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/output/pretrained_models/t5_ckpts/t5-v1_1-xxl
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior wi

In [ ]:
# !wandb login REPLACE_THIS_WITH_YOUR_AUTH_TOKEN_OF_WANDB

## Train model

In [6]:
%cd ../PixArt-alpha
!python -m torch.distributed.launch \
    train_scripts/train.py \
    /n/home12/binxuwang/Github/DiffusionObjectRelation/PixArt-alpha/notebooks/PixArt_xl2_img512_internal_for_pokemon_sample_training.py \
    --work-dir $STORE_DIR"/DL_Projects/PixArt/PixArt-alpha/output/trained_model" \
    --report_to="tensorboard" \
    --loss_report_name="train_loss"

/n/home12/binxuwang/Github/DiffusionObjectRelation/PixArt-alpha
/n/home12/binxuwang/.conda/envs/torch2/lib/python3.10/site-packages/torch/distributed/launch.py:183: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
[W socket.cpp:464] [c10d] The server socket cannot be initialized on [::]:29500 (errno: 97 - Address family not supported by protocol).
[W socket.cpp:697] [c10d] The client socket cannot be initialized to connect to [localhost]:29500 (errno: 97 - Address family not supported by protocol).
/n/home12/binxuwang/.conda/envs/torch2/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is dep